# London Underground Analysis

The London Underground is a network - and what better way to understand it and analyse it by using graph theory. In this notebook, I want to create the London Underground as a graph in python as attempt to find the sortest route visiting all stations (nodes).

In [2]:
import pandas as pd
import matplotlib as plt
import networkx as nx
import numpy as np

df = pd.read_csv('Inter Station Train Times.csv')

Now we have imported the data, we can clean it by naming the columns and dropping the columns just filled with NaN values. Then we will have a dataset of stations and distances between them to create our network with.

In [3]:
df.columns = df.iloc[0] # Re index columns to proper header
df.reindex(df.index.drop(0))
df = df.drop(0, axis = 0) # Drop reindedex row
df = df.dropna(axis = 1, how = 'all') # Drop columns with NaN values
df = df.dropna(axis = 0, how = 'all')
df.head()

,Line,Direction,Station from (A),Station to (B),Distance (Kms),Un-impeded Running Time (Mins),AM peak (0700-1000) Running Time (Mins),Inter peak (1000 - 1600) Running time (mins)
1,Metropolitan,Westbound,FINCHLEY ROAD,HARROW-ON-THE-HILL,11.63,11.02,11.00,11.00
2,Metropolitan,Eastbound,HARROW-ON-THE-HILL,FINCHLEY ROAD,11.63,10.15,11.50,11.50
3,Metropolitan,Eastbound,CHESHAM,CHALFONT & LATIMER,6.19,9.63,9.00,8.50
4,Metropolitan,Eastbound,MOOR PARK,HARROW-ON-THE-HILL,9.39,9.62,10.00,10.00
5,Metropolitan,Westbound,CHALFONT & LATIMER,CHESHAM,6.19,9.48,8.64,8.50


The data is quite good, and the best available for the job. However, it does need some cleaning so that we can use it for our purpose. Some stations that are one entity are listed separatly, for example there are 3 Paddington stations, PADDINGTION (CIRCLE), PADDINGTON (H&C) and PADDINGTON (DISTRICT). These all need to be merged to be one station, otherwise we have an inaccurate network for the TSP to solve.

We will use regex to solve this. We look for a space, followed by a bracket, followed by some characters then a closign bracket. This pattern is located and removed from the string, leaving just the station name in the dataframe. This will allow the add_edge function in networkx to build our graph

In [4]:
import re

for ix, item in df['Station from (A)'].iteritems(): # Get all items with brackets in and edit them, exepct for Kensington
    if "(" in str(item) and item != "KENSINGTON (OLYMPIA)":
        item = re.sub("\s[(][^)]+[)]", "", item) # Drop what is in brackets from end
        df.loc[ix]['Station from (A)'] = item # Put that back as the item

for ix, item in df['Station to (B)'].iteritems(): # Get all items with brackets in and edit them, exepct for Kensington
    if "(" in str(item) and item != "KENSINGTON (OLYMPIA)":
        item = re.sub("\s[(][^)]+[)]", "", item) # Drop what is in brackets from end
        df.loc[ix]['Station to (B)'] = item # Put that back as the item


In [5]:
g = nx.Graph()

for ix, connection in df.iterrows():
    station_from = connection['Station from (A)']
    station_to = connection['Station to (B)']
    
    # Add edge between two stations, then add data on which line each node is on
    g.add_edge(station_from, station_to, weight = float(connection['Un-impeded Running Time (Mins)']))
    
    # Add the information on what line the station is on
    
    try: #Node already has property "line"
        if connection['Line'] not in g.node[station_from]['Line']:
            g.node[station_from]['Line'].append(connection['Line'])
        else: continue
            
    except: #Node needs empty list created, and first station in it
        g.node[station_from]['Line'] = [connection['Line']]
        
        
    try: # Do the same for the other station in the edge
        if connection['Line'] not in g.node[station_to]['Line']:
            g.node[station_to]['Line'].append(connection['Line'])
        else: continue
            
    except:
        g.node[station_from]['Line'] = [connection['Line']]
        
# Manually add some paths

g.add_edge('KINGS CROSS', 'KINGS CROSS ST PANCRAS', weight = float(0.1))
g.add_edge('EALING BROADWAY', 'PADDINGTON', weight = float(11))
g.add_edge('UPMINSTER', 'TOWER HILL', weight = float(11))
g.add_edge('COCKFOSTERS', 'HIGH BARNET', weight = float (37)) # 384 Bus
g.add_edge('EDGWARE', 'STANMORE', weight = float(28)) #142 bus

print(nx.info(g))

Name: 
Type: Graph
Number of nodes: 269
Number of edges: 315
Average degree:   2.3420


In [6]:
print(nx.shortest_path(g, source = 'FINSBURY PARK', target = 'KINGS CROSS', weight = 'weight'))
print(nx.shortest_path_length(g, source='FINSBURY PARK', target='KINGS CROSS', weight='weight'))

['FINSBURY PARK', 'HIGHBURY', 'KINGS CROSS']
5.17


The graph knows the vic is fastest - the edge weighting has worked. Further, the TFL website says this should take 7 mins so we can see this data is accurate! 

Now we have the London Undergroud as a graph, with edge weights as times between stations. We can now analyse routes and begin to solve the problem. This problem is the travveling salesman problem, finding the shirtest time/distance to visit every node. This is an NP hard problem, so we can only find good approximations of a solution, not exact solutions.

We can find upper and lower bound to help us analyse our results. These are values than an optimal soultion will be inbetween.

An upper bound can be computed as twice the total weight of the minimum spanning tree. If we want to visit every node once, a minimum spanning tree describes this, but we get branches that might have to be traversed twice. A lower bound is therefore the size of the minimum spanning tree.

In [7]:
mst = nx.minimum_spanning_tree(g)
print("The upper bound is: ", 2*mst.size(weight = 'weight'), "mins")
print("The lower bound is: ", mst.size(weight = 'weight'), "mins")

The upper bound is:  1021.1600000000001 mins
The lower bound is:  510.58000000000004 mins


So we know that our solution should be between 510 and 1021 minutes.

Obviously, this value is not accounting for changing trains, waiting for trains and other issues along the way but these errors will be in all routes, so the end route should be ok. Also, we can later try to account for this in our modelling, by considering the time to change trains.

We can't attemot to find the optimal route, tryng every combination will be 269! stations, the eventual heat death of the universe will occur before we will get a solution. An improvement on this is the Held-Karp algorithm, but this is still an $\mathcal{O}(n\log{}n)$ algorithm - it still suffers from the same problem. For a network this large, we must use approximation algrothms.

The nearest neighbour algorithm is one to start with.
This takes the distance matrix of the whole network, starts at one node and then visits the closest unvisited node. This may produce a good tour, or may not. It will need improvement.

Names of data structures used:

mat - Matrix of times between stations, generated by using the Floyd-Warshall Algorithm

stations - list of stations, stations[i] gives the same station data as mat[i]

visited - list of station ids in the order that they are visited

route - list of station names in the order that they are visited

In [8]:
#Get a matrix of times between all stations
mat = nx.floyd_warshall_numpy(g, weight='weight')

In [9]:
stations = g.nodes() #ith column in matrix (mat) corresponds to ith station in g.nodes
visited = []

station = 23 #Start at Amersham-Chalfont in stations list
previous_station = 3
route = []

while len(set(visited)) < len(g.nodes())-1: # Set generates only unique items
    visited_on_route = nx.shortest_path(g, source = stations[previous_station], target = stations[station], weight = 'weight')[1:]

    for x in visited_on_route: # Mark all stations visited on route as visited
        visited.append(stations.index(x)) 
        route.append(x)
    
    row = mat[station]
    ordered_row = np.sort(row) # order by shortest distance
    
    for dist in np.nditer(ordered_row):
        if np.where(dist==row)[1][0] not in visited:
            previous_station = station
            station = np.where(dist==row)[1][0]
            break
            
print(route)

['AMERSHAM', 'CHALFONT & LATIMER', 'CHORLEYWOOD', 'RICKMANSWORTH', 'MOOR PARK', 'NORTHWOOD', 'NORTHWOOD HILLS', 'PINNER', 'NORTH HARROW', 'HARROW-ON-THE-HILL', 'NORTHWICK PARK', 'PRESTON ROAD', 'WEMBLEY PARK', 'NEASDEN', 'DOLLIS HILL', 'WILLESDEN GREEN', 'KILBURN', 'WEST HAMPSTEAD', 'FINCHLEY ROAD', 'SWISS COTTAGE', 'ST JOHNS WOOD', 'BAKER STREET', 'MARYLEBONE', 'EDGWARE ROAD', 'BAKER STREET ', 'GREAT PORTLAND STREET', 'EUSTON SQUARE', 'KINGS CROSS ST PANCRAS', 'KINGS CROSS', 'EUSTON', 'WARREN STREET', 'GOODGE STREET', 'TOTTENHAM COURT ROAD', 'OXFORD CIRCUS', 'BOND STREET', 'MARBLE ARCH', 'LANCASTER GATE', 'QUEENSWAY', 'NOTTING HILL GATE', 'HOLLAND PARK', 'SHEPHERDS BUSH', 'WHITE CITY', 'EAST ACTON', 'NORTH ACTON', 'WEST ACTON', 'EALING BROADWAY', 'EALING COMMON', 'ACTON TOWN', 'CHISWICK PARK', 'TURNHAM GREEN', 'STAMFORD BROOK', 'RAVENSCOURT PARK', 'HAMMERSMITH', 'BARONS COURT', 'WEST KENSINGTON', 'EARLS COURT', 'GLOUCESTER ROAD', 'SOUTH KENSINGTON', 'SLOANE SQUARE', 'VICTORIA', 'ST JA

In [10]:
# We can find the weight of this network, to see how long out route is
# We need 2 stations after each other (pattern [0,1],[1,2],[2,3]..., so the first iteration is impossible
# We store the previous station id number and then use the current one to get our reference.

total_weight = 0
previous_id = None

for station_id in visited:
    if previous_id == None:
        previous_id = station_id
    
    else:
        total_weight += mat[[previous_id],[station_id]]
        previous_id = station_id
         
print("The weight of this route is: ", total_weight, " mins")

The weight of this route is:  [[ 940.91]]  mins


We can implement the 2-opt swap on this network, to remove overlapping edges. 

The current distance is the sum of the two edges being considered. These are generated by looking up the distance in the distance matrix (mat) between every combination of edges in the network. 

These links explain the algorithm well:

https://stackoverflow.com/questions/33749268/generate-all-neighbors-for-2opt-in-python?rq=1

https://stackoverflow.com/questions/44487073/2-opt-algorithm-neighbourhood-of-a-given-tour

In [11]:
for i in range(0, len(visited)-3):
    for j in range(i+2, len(visited)-1):
        
        current_distance = mat[visited[i], visited[i+1]] + mat[visited[j], visited[j+1]]
        new_distance = mat[visited[i], visited[j]] + mat[visited[i+1], visited[j+1]]
        
        if current_distance > new_distance:
            
            # Create tempory values to change the route in the route list and visited list
            # These are the station ids
            old_i = [visited[i+1], route[i+1]] # List as old dist, old route string
            old_j = [visited[j], route[j]]
            
            visited[i+1] = old_j[0]
            route[i+1] = old_j[1]

            visited[j] = old_i[0]
            route[j] = old_i[1]
            

In [12]:
# Remove name duplicates next to each other
for ix in range(len(set(visited))):
    if ix == 0: # Skip first pass
        continue
    else:
        if visited[ix] == visited[ix-1]:
            del(visited[ix])
            del(route[ix])

In [13]:
print(route)

['AMERSHAM', 'CHALFONT & LATIMER', 'CHESHAM', 'CHALFONT & LATIMER', 'CHORLEYWOOD', 'CHORLEYWOOD', 'RICKMANSWORTH', 'RICKMANSWORTH', 'MOOR PARK', 'MOOR PARK', 'CROXLEY', 'WATFORD', 'CROXLEY', 'NORTHWOOD', 'NORTHWOOD HILLS', 'PINNER', 'NORTH HARROW', 'HARROW-ON-THE-HILL', 'HARROW-ON-THE-HILL', 'NORTHWICK PARK', 'PRESTON ROAD', 'WEMBLEY PARK', 'NEASDEN', 'DOLLIS HILL', 'WILLESDEN GREEN', 'KILBURN', 'WEST HAMPSTEAD', 'FINCHLEY ROAD', 'SWISS COTTAGE', 'ST JOHNS WOOD', 'BAKER STREET', 'GREAT PORTLAND STREET', 'EUSTON SQUARE', 'KINGS CROSS ST PANCRAS', 'FARRINGDON', 'BARBICAN', 'MOORGATE', 'OLD STREET', 'LIVERPOOL STREET', 'BETHNAL GREEN', 'MILE END', 'STRATFORD', 'LEYTON', 'LEYTONSTONE', 'SNARESBROOK', 'SOUTH WOODFORD', 'WOODFORD', 'BUCKHURST HILL', 'LOUGHTON', 'DEBDEN', 'THEYDON BOIS', 'EPPING', 'DEBDEN', 'BUCKHURST HILL', 'WOODFORD', 'SOUTH WOODFORD', 'SNARESBROOK', 'LEYTONSTONE', 'WANSTEAD', 'REDBRIDGE', 'GANTS HILL', 'NEWBURY PARK', 'BARKINGSIDE', 'FAIRLOP', 'HAINAULT', 'GRANGE HILL', 'C

In [14]:
#Again find the weight
total_weight = 0
previous_id = None

for station_id in visited:
    if previous_id == None:
        previous_id = station_id
    
    else:
        total_weight += mat[[previous_id],[station_id]]
        previous_id = station_id
         
print("The weight of this route is: ", total_weight, " mins")

The weight of this route is:  [[ 900.87]]  mins
